# CNN 학습 실습

## Import modules

In [2]:
# https://deep-eye.tistory.com/41

import tensorflow as tf

# tf.python.control_flow_ops = tf
import numpy as np
import pandas as pd
import sys

from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model

from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', None)

In [ ]:
# 사전 학습된 모델 불러오기
input_tensor = Input(shape=(150,150,3))
model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

# 모델 Layer 데이터화
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Layer 추가
x = layer_dict['block5_pool'].output
# Cov2D Layer +
x = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)
# MaxPooling2D Layer +
x = MaxPooling2D(pool_size=(2, 2))(x)
# Flatten Layer +
x = Flatten()(x)
# FC Layer +
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)

# new model 정의
new_model = Model(inputs = model.input, outputs = x)

In [ ]:
# CNN Pre-trained 가중치를 그대로 사용할때
for layer in new_model.layers[:19] :
    layer.trainable = False

new_model.summary()

# 컴파일 옵션
new_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

In [ ]:
# 모델 학습
history = new_model.fit(train_data_gen, epochs=5,
                        validation_data=test_data_gen)

## 하이퍼파라미터 정의

In [172]:
EPOCHS = 200

In [173]:
k = np.arange(12)

k = k.reshape(3,4,1)
k

array([[[ 0],
        [ 1],
        [ 2],
        [ 3]],

       [[ 4],
        [ 5],
        [ 6],
        [ 7]],

       [[ 8],
        [ 9],
        [10],
        [11]]])

## 네트워크 구조 정의

In [185]:
# mnist = input_data.read_data_sets('./MNIST_data/', one_hot=True)


np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', None)

# train_df = pd.read_csv(r'D:\대회\Mnist 손글씨 분류 경진대회\dataset\train\train_data.csv')
# train_dir = r'D:\대회\Mnist 손글씨 분류 경진대회\dataset\train\\'
train_df = pd.read_csv(r'C:\Users\6509504\Mnist_number\dataset\train\train_data.csv')
train_dir = r'C:\Users\6509504\Mnist_number\dataset\train\\'
train_file_path = train_dir + train_df['file_name']

    
def dataset_raw (path):
    
    image = Image.open(path) 
    image_array = np.array(image).reshape(28,28,1)#.reshape(784)
    return image_array

def dataset_rotation30 (path):
    
    rotation30 = transforms.RandomRotation(30)
    image = Image.open(path)
    image = rotation30(image)
    image_array = np.array(image).reshape(28,28,1)#.reshape(784)
    return image_array

def dataset_rotation40 (path):
    
    rotation30 = transforms.RandomRotation(30)
    image = Image.open(path)
    image = rotation30(image)
    image_array = np.array(image).reshape(28,28,1)#.reshape(784)
    return image_array


def dataset_crop (path):
    
    crop = transforms.CenterCrop(12)
    image = Image.open(path)
    image = crop(image)
    image_array = np.array(image).reshape(28,28,1) #.reshape(784)
    return image_array


def dataset_erase (path):
    
    erase = transforms.Compose([transforms.PILToTensor(),transforms.RandomErasing(0.5),transforms.ToPILImage() ])
    image = Image.open(path)
    image = erase(image)
    image_array = np.array(image).reshape(28,28,1)#.reshape(784)
    return image_array

# train_label = np.array(pd.get_dummies(train_df['label']))
train_label = np.array(train_df['label'])
print(train_label)

train_data_raw = np.array(list(train_file_path.apply(dataset_raw)), dtype='float32')
train_data_r30 = np.array(list(train_file_path.apply(dataset_rotation30)), dtype='float32')
train_data_r40 = np.array(list(train_file_path.apply(dataset_rotation40)), dtype='float32')
# train_data_crop = np.array(list(train_file_path.apply(dataset_crop)), dtype='float32')
train_data_erase = np.array(list(train_file_path.apply(dataset_erase)), dtype='float32')


train_data = np.vstack([train_data_raw, train_data_r30, train_data_r40, train_data_erase]) 
train_label = np.hstack([train_label, train_label, train_label, train_label]) 

[8 8 8 8 8 8 8 8 8 8 9 8 9 9 8 8 9 9 9 8 8 8 9 9 8 9 9 9 8 8 9 9 8 8 8 8 8
 8 8 8 9 8 9 8 9 9 8 8 9 8 8 8 8 9 9 9 9 9 9 9 8 8 9 9 8 9 8 8 8 9 9 9 9 8
 9 9 9 9 9 9 9 9 9 8 8 9 9 9 9 9 8 8 9 8 8 9 9 8 8 8 9 8 8 9 8 8 9 9 9 9 8
 8 9 8 8 9 8 9 9 8 8 9 8 9 9 8 9 8 8 8 8 8 9 9 8 9 8 8 8 9 9 8 8 9 8 8 8 9
 9 8 8 8 8 8 9 8 8 8 8 9 9 9 9 8 9 9 9 9 9 9 9 8 8 9 8 9 9 9 9 9 9 9 9 8 9
 8 9 9 9 9 9 8 8 9 8 8 9 8 9 8 9 9 9 2 8 2 8 2 8 8 8 9 8 9 8 2 8 8 2 8 9 2
 7 9 7 2 2 8 8 8 7 8 8 7 9 7 8 8 9 2 9 7 9 2 7 8 9 7 2 2 2 8 7 7 2 8 9 9 2
 3 3 7 2 8 8 9 3 2 2 2 7 7 2 2 3 9 7 8 7 8 8 2 2 7 9 2 7 3 9 9 3 9 7 8 9 8
 7 8 8 7 9 9 3 9 8 3 3 2 9 7 2 9 2 9 3 7 3 7 3 3 2 9 8 3 2 7 8 9 2 3 8 7 2
 9 9 9 9 8 9 8 8 8 7 9 9 7 3 2 2 3 3 2 7 9 8 9 7 8 9 3 3 2 8 8 9 7 3 7 8 2
 3 2 9 8 7 2 9 7 2 2 9 2 8 8 7 8 3 8 2 9 2 3 8 9 7 8 2 9 2 2 3 8 3 9 9 3 9
 7 3 2 9 2 8 9 7 7 7 8 7 2 7 9 7 2 9 2 3 9 2 2 7 3 7 7 7 2 8 7 9 9 7 7 2 8
 9 8 2 7 2 7 3 2 3 7 3 3 8 2 9 3 7 3 3 3 7 2 7 9 8 2 2 7 9 3 8 8 7 2 3 8 8
 2 2 2 2 2 2 2 7 2 7 8 8 

In [186]:
def MyModel():
    return Sequential([Conv2D(32, (3, 3), padding='same', activation='relu'), # 28x28x32
                       MaxPool2D(), # 14x14x32
                       Conv2D(64, (3, 3), padding='same', activation='relu'), # 14x14x64
                       MaxPool2D(), # 7x7x64
                       Conv2D(128, (3, 3), padding='same', activation='relu'), # 7x7x128
                       Flatten(), # 6272
                       Dense(128, activation='relu'),
                       Dense(10, activation='softmax')]) # 128

In [187]:
x_train = train_data.copy()
y_train = train_label.copy()
print(x_train.shape)
print(y_train.shape)

(20000, 28, 28, 1)
(20000,)


## 데이터 불러오기

In [188]:
# fashion_mnist = tf.keras.datasets.fashion_mnist

# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

# x_train = x_train.astype(np.float32)
# x_test = x_test.astype(np.float32)

# # NHWC
# x_train = x_train[..., np.newaxis]
# x_test = x_test[..., np.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
# test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

In [189]:
for a, b in train_ds.take(1):
    print(a,b)

tf.Tensor(
[[[[  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   [  0.]
   

In [190]:
# import matplotlib.pyplot as plt
# plt.imshow(fashion_mnist.load_data()[0][0][0])

## 모델 생성

In [191]:
model = MyModel()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 모델 학습

In [ ]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/200
625/625 [==============================] - 17s 27ms/step - loss: 0.4545 - accuracy: 0.9116 - val_loss: 2.3018 - val_accuracy: 0.1034
Epoch 2/200
625/625 [==============================] - 16s 26ms/step - loss: 0.0912 - accuracy: 0.9707 - val_loss: 2.3214 - val_accuracy: 0.1000
Epoch 3/200
625/625 [==============================] - 16s 26ms/step - loss: 0.0556 - accuracy: 0.9805 - val_loss: 2.5793 - val_accuracy: 0.1000
Epoch 4/200
625/625 [==============================] - 16s 25ms/step - loss: 0.0508 - accuracy: 0.9829 - val_loss: 3.3052 - val_accuracy: 0.1000
Epoch 5/200
625/625 [==============================] - 16s 25ms/step - loss: 0.0407 - accuracy: 0.9866 - val_loss: 5.2123 - val_accuracy: 0.1000
Epoch 6/200
625/625 [==============================] - 16s 25ms/step - loss: 0.0350 - accuracy: 0.9891 - val_loss: 9.1277 - val_accuracy: 0.1000
Epoch 7/200
625/625 [==============================] - 16s 26ms/step - loss: 0.0366 - accuracy: 0.9895 - val_loss: 6.1773 - val_ac

In [ ]:
# test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

# print('\nTest accuracy:', test_acc)

In [ ]:
test_df = pd.read_csv(r'C:\Users\6509504\Mnist_number\dataset\sample_submission.csv')
test_dir = r'C:\Users\6509504\Mnist_number\dataset\test\\'
test_file_path = test_dir + test_df['file_name']
print(test_file_path[0])

def dataset_raw (path):
    
    image = Image.open(path) 
    image_array = np.array(image).reshape(784)
    return image_array

test_data = np.array(list(test_file_path.apply(dataset_raw)))

In [141]:
a = test_data.reshape(5000, 28, 28)

In [142]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

predictions = probability_model.predict(a)


# model.predict(
#     x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
#     workers=1, use_multiprocessing=False
# )

In [167]:
for i in range(len(test_df['label'])):
    test_df['label'][i] = tf.math.argmax(predictions[i]).numpy()
    

C:\Users\6509504\Anaconda3\envs\tensorflow2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [169]:
test_df.to_csv('submit.csv', index=None)

In [154]:
predictions[3]

array([0.08533674, 0.08533674, 0.08533674, 0.23196931, 0.08533674,
       0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674],
      dtype=float32)

In [170]:
test_df

,file_name,label
0,idx0001.png,1
1,idx0002.png,0
2,idx0003.png,8
3,idx0004.png,3
4,idx0005.png,5
5,idx0006.png,9
6,idx0007.png,2
7,idx0008.png,4
8,idx0009.png,5
9,idx0010.png,1
